In [1]:
from vpolo.alevin import parser
import scanpy as sc
import pandas as pd
import os 
from scipy.io import mmwrite
from scipy.sparse import csr_matrix
import shutil

/opt/conda/envs/spatialdata/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/envs/spatialdata/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/opt/conda/envs/spatialdata/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)
/opt/conda/envs/spatialdata/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/opt/conda/envs/spatialdata/lib/python3.10/site-packages/anndata/utils.py:429: Fut

In [12]:

# takes alevin output files as input and provides with scanpy readable matrix.mtx, barcodes.tsv and genes.tsv files

Dataset_path = "/home/jovyan/ifbdata/spatial_cell_id/Kush/alignment/demo_matrix/align"
protein_coding_file = "/home/jovyan/ifbdata/spatial_cell_id/Reference/txp2gene/protein_coding_genes_version.txt"
Matrix_output = "/home/jovyan/ifbdata/spatial_cell_id/Kush/alignment/demo_matrix/matrix"
universal_genes_file = "/home/jovyan/ifbdata/spatial_cell_id/Reference/txp2gene/genes.tsv"

with open(protein_coding_file, "r") as file:
    protein_coding_genes = {line.strip() for line in file}

for subdir in os.listdir(Dataset_path):
    subdir_path = os.path.join(Dataset_path, subdir)
    if os.path.isdir(subdir_path):
        # print(subdir_path)
        alevin_df = parser.read_quants_bin(subdir_path) 
        # print(alevin_df.shape)
        filtered_alevin_df = alevin_df.T.loc[alevin_df.T.index.isin(protein_coding_genes)] #filtering with only protein coding genes
        # print(filtered_alevin_df.shape)
        missing_genes = list(set(protein_coding_genes) - set(alevin_df.T.index)) #missing genes in df
        missing_data = pd.DataFrame(0, index=missing_genes, columns=alevin_df.T.columns) #making missing df with 0 values for all cells
        # print(missing_data.shape)
        filtered_alevin_df = pd.concat([filtered_alevin_df, missing_data]) #adding missing with 0 values
        filtered_alevin_df = filtered_alevin_df.loc[protein_coding_genes] #order sorting
        # print(filtered_alevin_df.shape)
        adata = sc.AnnData(filtered_alevin_df)
        output = os.path.join(Matrix_output, subdir)
        if not os.path.exists(output):
            os.makedirs(output)
        # matrix file
        mmwrite(os.path.join(output, "matrix.mtx"), csr_matrix(adata.X))
        
        # barcode file
        barcodes_df = pd.DataFrame(adata.var_names) #not adata.obs_names as we are using transpose matrix
        barcodes_df.to_csv(os.path.join(output, "barcodes.tsv"), sep="\t", index=False, header=False)

        # since we are using universal genes.tsv file for all datasets, putting that file in every matrix folder
        # if it was not the universal order file, we could use similar as barcodes.tsv using adata.obs_names
        # gene file
        shutil.copy(universal_genes_file, os.path.join(output, "genes.tsv"))
    # break

/home/jovyan/ifbdata/spatial_cell_id/Kush/alignment/demo_matrix/align/SRR9036396/alevin
Using rust mode with 3281 rows and 78277 columns
(3281, 78277)
(20037, 3281)
(79, 3281)


/tmp/ipykernel_1226400/204657839.py:21: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  filtered_alevin_df = filtered_alevin_df.loc[protein_coding_genes] #order sorting


(20116, 3281)
/home/jovyan/ifbdata/spatial_cell_id/Kush/alignment/demo_matrix/align/SRR9036397/alevin
Using rust mode with 4544 rows and 78277 columns
(4544, 78277)
(20037, 4544)
(79, 4544)


/tmp/ipykernel_1226400/204657839.py:21: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  filtered_alevin_df = filtered_alevin_df.loc[protein_coding_genes] #order sorting


(20116, 4544)
